In [ ]:
import pandas as pd
import sqlite3
import os
import openai
from datetime import datetime
import json
from langchain.prompts import PromptTemplate
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

In [ ]:
# My variables 

working_directory = os.getcwd()
images_path = os.path.join(working_directory, 'images')
llm_model = "gpt-4-turbo"
db_name = "Database_transactions.db"
client = openai.OpenAI()

# Get today's date in YYYY-MM-DD format
today_date = datetime(2025, 3, 4).strftime("%Y-%m-%d")
day_of_week = datetime(2025, 3, 4).strftime("%A")

### Utils

In [ ]:
def load_image(image_name, dpi = 300, scale = 1.5):
    # Load an image
    image_path = os.path.join(images_path, image_name)
    
    img = mpimg.imread(image_path)
    height, width = img.shape[:2]  # Extract height & width

    fig_width = width / dpi  # Calculate figure width in inches
    fig_height = height / dpi  # Calculate figure height in inches

    plt.figure(figsize=(fig_width * scale, fig_height* scale), dpi=dpi)  

    # Display the image
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.show()

# Timing Decorator
def timing_decorator(func):
    """Decorator to measure execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Start timer
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # End timer
        execution_time = end_time - start_time
        #print(f"✅ {func.__name__} executed in {execution_time:.4f} seconds")
        return result, execution_time
    return wrapper

### Create database

In [ ]:
def create_db_sqlite(db_name=db_name):
    """
    Creates a SQLite database with a transactions table.
    If the table already exists, it is dropped and recreated with fresh data.
    """

    # Connect to SQLite database (creates the file if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute("DROP TABLE IF EXISTS transactions")
    cursor.execute("DROP TABLE IF EXISTS def_merchants")

    # Define the SQL query to create a new table
    create_transactions_table_query = """
    CREATE TABLE transactions (
        transaction_id TEXT PRIMARY KEY,
        date TEXT NOT NULL,
        amount REAL NOT NULL,
        merchant_id TEXT NOT NULL
    );
    """

    create_def_table_query = """
    CREATE TABLE def_merchants (
        merchant_id TEXT PRIMARY KEY,
        merchant TEXT NOT NULL,
        spending_cat TEXT NOT NULL
    );
    """

    # Execute the query to create the table
    cursor.execute(create_transactions_table_query)

    cursor.execute(create_def_table_query)

    # Define transaction data
    transactions = [
        ('T-00', '2023-06-15', 35.0, "M-01"),
        ('T-01', '2023-07-20', 50.0, "M-04"),
        ('T-02', '2023-09-05', 75.0, "M-05"),
        ('T-03', '2024-08-01', 43, "M-07"),
        ('T-04', '2024-08-01', 45.0, "M-03"),
        ('T-05', '2024-08-31', 49, "M-02"),
        ('T-06', '2024-08-31', 50.0, "M-03"),
        ('T-07', '2024-09-30', 55, "M-01"),
        ('T-08', '2024-09-30', 55.0, "M-04"),
        ('T-09', '2024-10-30', 61, "M-01"),
        ('T-10', '2024-10-30', 60.0, "M-03"),
        ('T-11', '2024-11-29', 67, "M-01"),
        ('T-12', '2024-11-29', 65.0, "M-03"),
        ('T-13', '2024-12-29', 73, "M-02"),
        ('T-14', '2024-12-29', 70.0, "M-03"),
        ('T-15', '2025-01-28', 79, "M-01"),
        ('T-16', '2025-01-28', 75.0, "M-06"),
        ('T-17', '2025-02-27', 85, "M-07"),
        ('T-18', '2025-02-27', 80.0, "M-03"),
        ('T-19', '2025-03-01', 20.0, "M-04"),
        ('T-20', '2025-03-01', 10.0, "M-01"),
        ('T-21', '2025-03-01', 40.0, "M-06"),
        ('T-22', '2025-03-02', 10.0, "M-01"),
        ('T-23', '2025-03-03', 60.0, "M-05"),
        ('T-24', '2025-03-03', 100.0, "M-04"),
        ('T-25', '2025-03-04', 40.0, "M-08"),
        ('T-26', '2025-03-04', 10.0, "M-06")]

    def_merchants = [("M-01", "TESCO", "GROCERIES"), 
                 ("M-02", "WAITROSE", "GROCERIES"), 
                 ("M-03", "VUE", "ENTERTAINMENT"), 
                 ("M-04", "STARLING", "FINANCE"), 
                 ("M-05", "SKY", "BILLS"), 
                 ("M-06", "MCDONALD'S", "EATING OUT"),
                 ("M-07", "STARBUCKS", "COFFEE"),
                  ("M-08", "MONZO", "FINANCE")]

    # Insert transaction data into the table
    cursor.executemany("INSERT INTO transactions VALUES (?, ?, ?, ?)", transactions)

    cursor.executemany("INSERT INTO def_merchants VALUES (?, ?, ?)", def_merchants)

    # Commit changes and close the connection
    conn.commit()

    print(f"Database '{db_name}' created successfully with 'transactions' table rewritten.")

    # Create a cursor object
    #cursor = conn.cursor()
    
    # Define SQL query to fetch all transactions
    #query = "SELECT merchant_id, merchant, spending_cat FROM def_merchants ;"
    query = "SELECT transaction_id, date, amount, spending_cat, merchant FROM transactions LEFT JOIN def_merchants ON transactions.merchant_id = def_merchants.merchant_id ;"

    # Execute the query
    cursor.execute(query)

    # Fetch all results
    transactions = cursor.fetchall()
    first_transaction_date = transactions[0][1]
    # Print results
    print("\n\n\nDisplaying table: \n")
    #print("Merchant ID | Merchant       |Spending Category")  
    print("Transaction ID | Date       | Amount | Spending Category | Merchant")  
    print("-" * 70)
    for transaction in transactions:
        #print(f"{transaction[0]:<11} | {transaction[1]:<14} | {transaction[2]:<6}")
        print(f"{transaction[0]:<14} | {transaction[1]:<10} | {transaction[2]:<6} | {transaction[3]:<18}| {transaction[4]:<10}")

    # Close the connection
    conn.close()
    return first_transaction_date

### Create a SQLite database with a transactions table

In [ ]:
first_transaction_date = create_db_sqlite()

### Constants

In [ ]:
# Predefined spending categories and merchants for normalization
CATEGORY_MAPPING = {
    "GROCERIES": ["food shop", "supermarket", "shopping"],
    "ENTERTAINMENT": ["movies", "cinema", "fun", "leisure"],
    "EATING OUT": ["restaurant", "dining", "takeaway"],
    "COFFEE": ["cafe", "starbucks", "coffee shop"],
    "FINANCE": ["bank", "investment"],
    "BILLS": ["utilities", "electricity", "internet"]
}

function_calling = [
    {
        "name": "extract_query_features",
        "description": "Extracts key features from a user query related to transactions, including date ranges (formatted as YYYY-MM-DD), merchant names, spending categories, and the required SQL aggregation function.",
        "parameters": {
            "type": "object",
            "properties": {
                "start_date": {
                    "type": "string",
                    "format": "date",
                    "description": f"""The start date extracted from the query (YYYY-MM-DD). If the query specifies a relative time frame (e.g., 'last week', 'last 4 months'), compute the actual date based on today's date  (Today is {today_date}, {day_of_week}).
                                    If the query does not specify a start date, use the earliest date available in the database, namely {first_transaction_date}. the week starts on Monday and ends on Sunday. If the date format is ambiguous, assume it follows the DD-MM-YYYY format"""
                },
                "end_date": {
                    "type": "string",
                    "format": "date",
                    "description": "The end date extracted from the query (YYYY-MM-DD). If the query specifies a relative time frame, compute the actual date based on today's date."
                },
                "merchant": {
                    "type": "string",
                    "nullable": True,
                    "description": """The merchant name mentioned in the query, if present. For example, "McDonald's". If no merchant is specified, this should be null."""
                },
                "spending_category": {
                    "type": "string",
                    "enum": ["GROCERIES", "ENTERTAINMENT", "EATING OUT", "COFFEE", "FINANCE", "BILLS"],
                    "nullable": True,
                    "description": "The spending category mentioned in the query, if present. If no category is mentioned but only a merchant, this must be null."
                },
                "sql_aggregation": {
                    "type": "string",
                    "enum": ["SUM", "AVG", "COUNT", "MAX", "MIN"],
                    "description": "The SQL aggregation function required to answer the query. Example: 'SUM' for total spend, 'COUNT' for the number of transactions."
                }
            },
            "required": ["start_date", "end_date", "sql_aggregation"]
        }
    }
]

In [ ]:
@timing_decorator
def preprocess_user_query(user_query):

    
    """
    Uses OpenAI to correct spelling, normalize language, and make the query more structured.
    """

    prompt_preprocess = PromptTemplate.from_template("""
    Given a user query about their transactions, perform these two tasks to preprocess the query for further analysis:
                                                     
        Task 1: correct spelling and grammar while ensuring key financial terms remain intact. If a merchant name is misspelled, replace with the correct spelling.
                Example input: "how muc i spnd at fod sop at Weitros in lastt 3 months?"
                Example output: "How much did I spend at food shop at Waitrose in the last 3 months?"

                Example input: "how much i spnd at Mcdonald two month go?"
                Example output: "How much did I spend at McDonald's two months ago?"
                                                     
        Task 2: if a spending category is mentioned, normalize it to a standard category.
                Use this mapping structure to map user input to **ONLY** these standard categories GROCERIES, ENTERTAINMENT, EATING OUT, COFFEE, FINANCE, BILLS. 
                {CATEGORY_MAPPING}
                
                NOTE that the values of the provided mapping are not exhaustive lists of synonyms for each standard categories.

                Example input:  "How much did I spend at food shop at Waitrose in the last 3 months?"
                Example output: "How much did I spend in GROCERIES at Waitrose in the last 3 months?"
                                                     
                Example input:  "How much did I spend for brunch from 01/12/2024 to 01/01/2025?"
                Example output: "How much did I spend in EATING OUT for brunch from 01/12/2024 to 01/01/2025?"
    
    User query: "{user_query}"
                                                     
    Return only the query after processing the above tasks. Do not include any other text in your answer
    Normalized query:
    """)

    prompt = prompt_preprocess.format( user_query=user_query, CATEGORY_MAPPING = CATEGORY_MAPPING)

    response = client.chat.completions.create(
        model = llm_model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()


def generate_sql_query(start_date, end_date, merchant, spending_category, sql_aggregation):

    base_query = f"""
    WITH transactions_view AS (
    SELECT transaction_id, date, amount, def_merchants.spending_cat, def_merchants.merchant FROM transactions LEFT JOIN def_merchants ON transactions.merchant_id = def_merchants.merchant_id)
    
    SELECT {sql_aggregation}(amount) AS total_spending
    FROM transactions_view
    WHERE date BETWEEN ? AND ?
    """

    query_params = [start_date, end_date]

    # Add merchant filter if provided
    if merchant:
        base_query += " AND merchant = ?"
        query_params.append(merchant.upper())

    # Add spending category filter if provided
    if spending_category:
        base_query += " AND spending_cat = ?"
        query_params.append(spending_category.upper())
    #print(base_query)
    return base_query, query_params  # Return query string and parameters


def call_database(sql_query, query_params):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(sql_query, query_params)

        # Fetch all results
        results = cursor.fetchall()

        # Commit and close connection
        conn.commit()
        conn.close()
        #print(results)
        return results  # Return the fetched query results

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

@timing_decorator
def function_calling_db(user_query):
    client = openai.OpenAI()

    # Get today's date in YYYY-MM-DD format
    today_date = datetime(2025, 3, 4).strftime("%Y-%m-%d")
    day_of_week = datetime(2025, 3, 4).strftime("%A")
    
    # Append today's date to the user query

    preprocessed_user_query, preprocess_time = preprocess_user_query(user_query)

    prompt_enhanced = PromptTemplate.from_template("{preprocessed_user_query} (Today is {today_date}, {day_of_week})")

    prompt = prompt_enhanced.format( preprocessed_user_query = preprocessed_user_query, today_date=today_date, day_of_week=day_of_week)

    # Call OpenAI API with function calling
    response = client.chat.completions.create(
        model=llm_model,
        messages=[{"role": "system", "content": "You extract parameters from a user query"},
                  {"role": "user", "content": prompt}],
        functions = function_calling,
        function_call="auto"
    )
    
    # Check if function calling response is present
    if response.choices[0].message.function_call:
        function_args = json.loads(response.choices[0].message.function_call.arguments)
        #print(function_args)

        # Extract required parameters
        start_date = function_args.get("start_date")
        end_date = function_args.get("end_date")
        merchant = function_args.get("merchant")
        if merchant:
            spending_category = None
            merchant = merchant.upper()
        else:
            spending_category = function_args.get("spending_category")
            if spending_category:
                spending_category = spending_category.upper()
        sql_aggregation = function_args.get("sql_aggregation")


        # Call SQL query generator
        sql_query, query_params = generate_sql_query(start_date, end_date, merchant, spending_category, sql_aggregation)

        #print(sql_query, query_params)
        db_results = call_database(sql_query, query_params)
        if db_results:
            result = db_results[0][0]
        else:
            result = None

        return preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, result, preprocess_time


    return "No function call detected, unable to generate SQL query."


In [ ]:
load_image('test_set.png', dpi = 300, scale = 1)

2) design the process

In [ ]:
load_image('process_design.png', dpi = 300, scale = 1.5)

In [ ]:
user_query = "What was the total spend at McDonald's last week?"


(full_result, function_time) = function_calling_db(user_query)
preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, db_results, preprocess_time = full_result

In [ ]:
print(preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, db_results)

In [ ]:
user_queries = ["How much did I spend on groceries in the last 4 months?",
                "What was the total spend at McDonald's last week?", 
                "What was the max I have spent on groceries in the last year?",
                "How much did I spend on cinemas in the last year?",
                "wht was my spending on fod in the last 3 months?",  # spelling mistakes and normalization
                "how much did I invest in starling since January 1st, 2025?", # alternative date formatting
                "how much did I invest since January 1st, 2025?",  # test that filters only by FINANCE 
                "how many times I went to the cinema in the last 3 months?", # test the count
                "What was the total spend at Tesco?",   # test that you it works without a start date
                "What was the average spending at Weitrose?", # test the average and the spelling of a brand
                "What was the overall spending at Starling between 01/03/2025 and 11/03/2025?", # testing the ambiguous date format
                "What was the average spending at Tesco between 13/01/2025 and 20/04/2025?", # testing the date range (DD-MM-AAAA)
                 ]   

In [ ]:
# Data storage for results
results_data = []

# Run function_calling_db for each query
for user_query in user_queries:
    start_time = time.time()  # Start timing

    (full_result, function_time) = function_calling_db(user_query)
    preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, db_results, preprocess_time = full_result
    try:
        db_results = round(db_results,2)
    except:
        pass
              

    end_time = time.time()  # End timing
    total_execution_time = end_time - start_time  # Calculate execution time

    # Store results
    results_data.append({
        "Original Query": user_query,
        "Cleaned Query": preprocessed_user_query,  # Store cleaned version
        "start_date": start_date,
        "end_date": end_date,
        "merchant": merchant,
        "spending_category": spending_category,
        "SQL_aggregation": sql_aggregation,
        "Response": db_results,
        "Preprocessing time (s)": preprocess_time,
        "Total Time (s)": round(total_execution_time, 4)
        }
    )

# Convert results to DataFrame
df_results = pd.DataFrame(results_data)

In [ ]:
df_results.to_csv("results.csv", index=False)